# Image Reconstruction

In [ ]:
# This cells setups the environment when executed in Google Colab.
try:
    import google.colab
    !curl -s https://raw.githubusercontent.com/ibs-lab/cedalion/dev/scripts/colab_setup.py -o colab_setup.py
    # Select branch with --branch "branch name" (default is "dev")
    %run colab_setup.py
except ImportError:
    pass

## Notebook configuration 
Decide for an example dataset with a sparse probe or a high-density probe for DOT. 
The notebook will load example data accordingly.

Also specify, if precomputed results of the photon propagation should be used and
if the 3D visualizations should be interactive.

In [ ]:
# choose between two datasets
DATASET = "fingertappingDOT" # high-density montage
#DATASET = "fingertapping"   # sparse montage

# choose a head model
HEAD_MODEL = "colin27"
# HEAD_MODEL = "icbm152"

# choose between the monte
FORWARD_MODEL = "MCX" # photon monte carlo
#FORWARD_MODEL = "NIRFASTER" # finite element method - NOTE, you must have NIRFASTer installed via runnning <$ bash install_nirfaster.sh CPU # or GPU> from a within your cedalion root directory.

# set this flag to False to actual compute the forward model results
PRECOMPUTED_FLUENCE = True

# set this flag to True to enable interactive 3D plots
INTERACTIVE_PLOTS = False

In [ ]:
import pyvista as pv

if INTERACTIVE_PLOTS:
    pv.set_jupyter_backend('server')
else:
    pv.set_jupyter_backend('static')

import os
import time
import warnings
from pathlib import Path
from tempfile import TemporaryDirectory

import matplotlib.pyplot as p
import numpy as np
import xarray as xr
from IPython.display import Image
from pint.errors import UnitStrippedWarning

import cedalion
import cedalion.dataclasses as cdc
import cedalion.datasets
import cedalion.geometry.registration
import cedalion.geometry.segmentation
import cedalion.imagereco.forward_model as fw
import cedalion.imagereco.tissue_properties
import cedalion.io
import cedalion.plots
import cedalion.sigproc.quality as quality
import cedalion.sigproc.motion_correct as motion_correct
import cedalion.vis.plot_sensitivity_matrix
from cedalion import units
from cedalion.imagereco.solver import pseudo_inverse_stacked
from cedalion.io.forward_model import FluenceFile, load_Adot

import cedalion.xrutils as xrutils
xrutils.unit_stripping_is_error()


xr.set_options(display_expand_data=False);

In [ ]:
# helper function to display gifs in rendered notbooks
def display_image(fname : str):
    display(Image(data=open(fname,'rb').read(), format='png'))

## Working Directory
In this notebook the output of the fluence and sensitivity calculations are stored in a temporary directory. This will be 
deleted when the notebook ends.

In [ ]:
temporary_directory = TemporaryDirectory()
tmp_dir_path = Path(temporary_directory.name)

## Load a finger-tapping dataset 

For this demo we load an example finger-tapping recording through either `cedalion.datasets.get_fingertapping` or `cedalion.datasets.get_fingertappingDOT`. 

The snirf files of these datasets contains a single NIRS element with one block of raw amplitude data.

In [ ]:
if DATASET == "fingertappingDOT":
    rec = cedalion.datasets.get_fingertappingDOT()
elif DATASET == "fingertapping":
    rec = cedalion.datasets.get_fingertapping()
else:
    raise ValueError("unknown dataset")

The location of the probes is obtained from the snirf metadata (i.e. /nirs0/probe/)

Note that units ('m') are adopted and the coordinate system is named 'digitized'.

In [ ]:
geo3d_meas = rec.geo3d
display(geo3d_meas)

In [ ]:
# visualize the montage
cedalion.plots.plot_montage3D(rec["amp"], geo3d_meas)

The measurement list is a `pandas.DataFrame` that describes which source-detector pairs form channels.

In [ ]:
meas_list = rec._measurement_lists["amp"]
display(meas_list.head(5))

Event/stimulus information is also stored in a `pandas.DataFrame`. 

In [ ]:
rec.stim

For clarity, events are given more descriptive names:

In [ ]:

if DATASET == "fingertappingDOT":
   rec.stim.cd.rename_events( {
        "1": "Control", 
        "2": "FTapping/Left", 
        "3": "FTapping/Right",
        "4": "BallSqueezing/Left",
        "5": "BallSqueezing/Right"
    } )
elif DATASET == "fingertapping":
    rec.stim.cd.rename_events( {
        "1.0": "Control", 
        "2.0": "FTapping/Left", 
        "3.0": "FTapping/Right"
    } )

# count number of trials per trial_type
display(
    rec.stim.groupby("trial_type")[["onset"]]
    .count()
    .rename({"onset": "#trials"}, axis=1)
)

## Preprocessing
Perform motion correction, conversion to optical density and bandpass filtering.

In [ ]:
rec["od"] = cedalion.nirs.int2od(rec["amp"])
rec["od_tddr"] = motion_correct.tddr(rec["od"])
rec["od_wavelet"] = motion_correct.wavelet(rec["od_tddr"])

# bandpass filter the data
rec["od_freqfiltered"] = rec["od_wavelet"].cd.freq_filter(
    fmin=0.01, fmax=0.5, butter_order=4
)

## Calculate block averages in optical density

In [ ]:
# segment data into epochs
epochs = rec["od_freqfiltered"].cd.to_epochs(
    rec.stim,  # stimulus dataframe
    ["FTapping/Left", "FTapping/Right"],  # select fingertapping events, discard others
    before=5 * units.s,  # seconds before stimulus
    after=30 * units.s,  # seconds after stimulus
)

# calculate baseline
baseline = epochs.sel(reltime=(epochs.reltime < 0)).mean("reltime")

# subtract baseline
epochs_blcorrected = epochs - baseline

# group trials by trial_type. For each group individually average the epoch dimension
blockaverage = epochs_blcorrected.groupby("trial_type").mean("epoch")

# Plot block averages. Please ignore errors if the plot is too small in the HD case

noPlts2 = int(np.ceil(np.sqrt(len(blockaverage.channel))))
f,ax = p.subplots(noPlts2,noPlts2, figsize=(12,10))
ax = ax.flatten()
for i_ch, ch in enumerate(blockaverage.channel):
    for ls, trial_type in zip(["-", "--"], blockaverage.trial_type):
        ax[i_ch].plot(blockaverage.reltime, blockaverage.sel(wavelength=760, trial_type=trial_type, channel=ch), "r", lw=2, ls=ls)
        ax[i_ch].plot(blockaverage.reltime, blockaverage.sel(wavelength=850, trial_type=trial_type, channel=ch), "b", lw=2, ls=ls)

    ax[i_ch].grid(1)
    ax[i_ch].set_title(ch.values)
    ax[i_ch].set_ylim(-.02, .02)
    ax[i_ch].set_axis_off()
    ax[i_ch].axhline(0, c="k")
    ax[i_ch].axvline(0, c="k")

p.suptitle("760nm: r | 850nm: b | left: - | right: --")
p.tight_layout()


## Load segmented MRI scan

For this example use a segmentation of the Colin27 average brain.

In [ ]:
if HEAD_MODEL == "colin27":
    SEG_DATADIR, mask_files, landmarks_file = cedalion.datasets.get_colin27_segmentation()
    PARCEL_DIR = cedalion.datasets.get_colin27_parcel_file()
elif HEAD_MODEL == "icbm152":
    SEG_DATADIR, mask_files, landmarks_file = cedalion.datasets.get_icbm152_segmentation()
    PARCEL_DIR = cedalion.datasets.get_icbm152_parcel_file()
else:
    raise ValueError("unknown head model")

The segmentation masks are in individual niftii files. The dict `mask_files` maps mask filenames relative to `SEG_DATADIR` to short labels. These labels describe the tissue type of the mask. 

In principle the user is free to choose these labels. However, they are later used to lookup the tissue's optical properties. So they must be map to one of the tabulated tissue types (c.f. `cedalion.imagereco.tissue_properties.TISSUE_LABELS`).

The variable `landmarks_file` holds the path to a file containing landmark positions in scanner space (RAS). This file can be created with Slicer3D.

In [ ]:
display(SEG_DATADIR)
display(mask_files)
display(landmarks_file)

## Coordinate systems

Up to now we have geometrical data from three different coordinate reference systems (CRS):

- The optode positions are in one space `CRS='digitized'` and the coordinates are in meter. In our example the origin is at the head center and y-axis pointing in the superior direction. Other digitization tools can use other units or coordinate systems.
- The segmentation masks are in voxel space (`CRS='ijk'`) in which the voxel edges are aligned with the coordinate axes. Each voxel has unit edge length, i.e. coordinates are dimensionless. 
  Axis-aligned grids are computationally efficient, which is why the photon simulation code (MCX) uses this coordinate system.
- The voxel space (`CRS='ijk'`) is related to scanner space (`CRS='ras'` or `CRS='aligned'`) in which coordinates have physical units and coordinate axes point to the (r)ight, (a)nterior and s(uperior) directions. The relation between both spaces is given through an affine transformation (e.g. `t_ijk2ras`). When loading the segmentation masks in Slicer3D this transformation is automatically applied. Hence, the picked landmark coordinates are exported in RAS space.

  The niftii file provides a string label for the scanner space. In this example the RAS space is called 'aligned' because the masks are aligned to another MRI scan.


To avoid confusion between these different coordinate systems, `cedalion` tries to be explicit about which CRS a given point cloud or surface is in.

## The TwoSurfaceHeadModel

The photon propagation considers the complete MRI scan, in which each voxel is attributed to one tissue type with its respective optical properties. However, the image reconstruction does not intend to reconstruct absorption changes in each voxel. The inverse problem is simplified, by considering only two surfaces (scalp and brain) and reconstruct only absorption changes in voxels close to these surfaces.

The class `cedalion.imagereco.forward_model.TwoSurfaceHeadModel` groups together the segmentation mask, landmark positions and affine transformations as well as the scalp and brain surfaces. The brain surface is calculated by grouping together white and gray matter masks. The scalp surface encloses the whole head.

In [ ]:
head = fw.TwoSurfaceHeadModel.from_surfaces(
    segmentation_dir=SEG_DATADIR,
    mask_files = mask_files,
    brain_surface_file= os.path.join(SEG_DATADIR, "mask_brain.obj"),
    scalp_surface_file= os.path.join(SEG_DATADIR, "mask_scalp.obj"),
    landmarks_ras_file=landmarks_file,
    parcel_file=PARCEL_DIR,
    brain_face_count=None,
    scalp_face_count=None
)

In [ ]:
head.segmentation_masks

In [ ]:
head.landmarks

In [ ]:
head.brain

In [ ]:
head.scalp

`TwoSurfaceHeadModel.from_surfaces` converts everything into voxel space (`CRS='ijk'`)

In [ ]:
head.crs

The transformation matrix to translate from voxel to scanner space:

In [ ]:
head.t_ijk2ras

Changing between coordinate systems:

In [ ]:
head_ras = head.apply_transform(head.t_ijk2ras)
display(head_ras.crs)
display(head_ras.brain)

## Optode Registration
The optode coordinates from the recording must be aligned with the scalp surface. Currently, `cedaĺion` offers a simple registration method, which finds an affine transformation (scaling, rotating, translating) that matches the landmark positions of the head model and their digitized counter parts. Afterwards, optodes are snapped to the nearest vertex on the scalp.

In [ ]:
geo3d_snapped_ijk = head.align_and_snap_to_scalp(geo3d_meas)
display(geo3d_snapped_ijk)

In [ ]:
plt = pv.Plotter()
cedalion.plots.plot_surface(plt, head.brain, color="w")
cedalion.plots.plot_surface(plt, head.scalp, opacity=.1)
cedalion.plots.plot_labeled_points(plt, geo3d_snapped_ijk)
plt.show()

## Simulate light propagation in tissue

`cedalion.imagereco.forward_model.ForwardModel` is a wrapper around pmcx. Using the data in the head model it prepares the inputs for either pmcx or NIRFASTer and offers functionality to calculate the sensitivty matrix.

In [ ]:
fwm = cedalion.imagereco.forward_model.ForwardModel(head, geo3d_snapped_ijk, meas_list)

### Run the simulation

The `compute_fluence_mcx` and `compute_fluence_nirfaster` methods simulate a light source at each optode position and calculate the fluence in each voxel. By setting `RUN_PACKAGE`, you can choose between the pmcx or NIRFASTer package to perform this simulation.
PLEASE NOTE: if you USE_CACHED data (download the example data) be aware that the file is quite big (~2GB).

In [ ]:
if PRECOMPUTED_FLUENCE:
    if FORWARD_MODEL == "MCX":
        fluence_fname = cedalion.datasets.get_precomputed_fluence(DATASET, HEAD_MODEL)
    elif FORWARD_MODEL == "NIRFASTER":
        raise NotImplementedError(
            "Currently there are no precomputed NIRFASTER results available"
        )
else:
    fluence_fname = tmp_dir_path / "fluence.h5"

    if FORWARD_MODEL == "MCX":
        fwm.compute_fluence_mcx(fluence_fname)
    elif FORWARD_MODEL == "NIRFASTER":
        fwm.compute_fluence_nirfaster(fluence_fname)

The photon simulation yields the fluence in each voxel for each wavelength:

- `fluence_all` is a `xr.DataArray` with dimensions: ('label', 'wavelength', 'i', 'j', 'k'),

   i.e. for each optode and wavelength it stores the 3D image of the computed fluence in each voxel

- `fluence_at_optodes` is a `xr.DataArray` with dimensions: ('optode1', 'optode2', 'wavelength').

  It contains the fluence directly at the position of the optodes, used for normalization purposes.


Both arrays are stored on disk in the hdf5 file at `fluence_fname` and should be queried through `cedalion.io.forward_model.FluenceFile`.

Also, for a each combination of two optodes, the fluence in the voxels at the optode positions is calculated.

## Plot fluence

To illustrate the tissue probed by light travelling from a source to the detector two fluence profiles need to be multiplied. 

In [ ]:
# for plotting use a geo3d without the landmarks
geo3d_plot = geo3d_snapped_ijk[geo3d_snapped_ijk.type != cdc.PointType.LANDMARK]

In [ ]:
time.sleep(1)

plt = pv.Plotter()

if DATASET == "fingertappingDOT":
    src, det, wl = "S5", "D16", 760
elif DATASET == "fingertapping":
    src, det, wl = "S2", "D3", 760
else:
    raise ValueError("unknown dataset")

# fluence_file.get_fluence returns a 3D numpy array with the fluence
# for a specified source and wavelength.
with FluenceFile(fluence_fname) as fluence_file:
    f = fluence_file.get_fluence(src, wl) * fluence_file.get_fluence(det, wl)

f[f <= 0] = f[f > 0].min()
f = np.log10(f)
vf = pv.wrap(f)

plt.add_volume(
    vf,
    log_scale=False,
    cmap="plasma_r",
    clim=(-10, 0),
)
cedalion.plots.plot_surface(plt, head.brain, color="w")
cedalion.plots.plot_labeled_points(plt, geo3d_plot, show_labels=False)

cog = head.brain.vertices.mean("label")
cog = cog.pint.dequantify().values
plt.camera.position = cog + [-300, 30, 100]
plt.camera.focal_point = cog
plt.camera.up = [0, 0, 1]

plt.show()

### Calculate the sensitivity matrices
The forward model's function `compute_sensitivity` calculates the sensitivity matrix from the fluence file and saves the result in a new file.

In [ ]:
sensitivity_fname = tmp_dir_path / "sensitivity.h5"
fwm.compute_sensitivity(fluence_fname, sensitivity_fname)

The sensitivity matrix describes how an absorption change at a given surface vertex changes the optical density in a given channel and wavelength. 

The coordinate `is_brain` holds a mask to distinguish brain and scalp voxels.

In [ ]:
# load and display sensitivity matrix
Adot = load_Adot(sensitivity_fname)
display(Adot)

### Plot Sensitivity Matrix

In [ ]:
plotter = cedalion.vis.plot_sensitivity_matrix.Main(
    sensitivity=Adot,
    brain_surface=head.brain,
    head_surface=head.scalp,
    labeled_points=geo3d_plot,
)
plotter.plot(high_th=0, low_th=-3)
plotter.plt.show()

The sensitivity `Adot` has shape (nchannel, nvertex, nwavelenghts). To solve the inverse problem we need a matrix that relates OD in channel space to concentration changes in image space. Hence, the sensitivity must include the extinction coefficients to translate between OD and concentrations. Furthermore, channels at different wavelengths and vertices at different chromophores  must be  stacked into new dimensions (called `'flat_channel'` and `'flat_vertex'`) to yield the following matrix equation:

$$ \left( \begin{matrix} OD_{c_1, \lambda_1} \\ \vdots \\ OD_{c_N,\lambda_1} \\ OD_{c_1,\lambda_2} \\ \vdots \\ OD_{c_N,\lambda_2} \end{matrix}\right) = A \cdot
\left( \begin{matrix} \Delta c_{v_1, HbO} \\ \vdots \\ \Delta c_{v_N, HbO} \\ \Delta c_{v_1, HbR} \\ \vdots \\ \Delta c_{v_N, HbR} \end{matrix}\right) $$

In [ ]:
Adot_stacked = fwm.compute_stacked_sensitivity(Adot)
Adot_stacked

### Invert the sensitivity matrix

With `'Adot_stacked'` now being a 2D matrix, its pseudo-inverse can be computed with the function `pseude_inverse_stacked`. Different regularization options are implemented and can be controled through the parameters `alpha`, `alpha_spatial` and `Cmeas`.

In [ ]:
B = pseudo_inverse_stacked(Adot_stacked, alpha = 0.01, alpha_spatial = 0.001)
B

### Calculate concentration changes

- the optical density has shape (nchannel, nwavelength, time). Additional dimensions like 'trial_type' in this example are allowed, too.

In [ ]:
blockaverage

To apply the inverted sensitiviy matrix, the OD wavelength and channel dimensions need to be stacked. Then 
the inverted sensitivity matrix can be multiplied which contracts over `'flat_channel'` and the `'flat_vertex'` dimension remains.
The `'flat_vertex'` dimensions contains vertices of the scalp and the brain for both chromophores. These need to be
separated. The function `fw.apply_inv_sensitiviy` takes care of all of this. 

In [ ]:
dC_brain, dC_scalp = fw.apply_inv_sensitivity(blockaverage, B)
display(dC_brain)
display(dC_scalp)

Convert concentration changes into micromolar. 

In [ ]:
dC_brain = dC_brain.pint.quantify().pint.to("uM").pint.dequantify()
dC_scalp = dC_scalp.pint.quantify().pint.to("uM").pint.dequantify()

## Visualizing image reconstruction results

In the following, different cedalion plot functions will be showcased to visualize concentration changes on the brain and scalp surfaces.

### Channel Space

The function `cedalion.plots.scalp_plot_gif` allows to create an animated gif of channel-space OD changes projected on the scalp.

Here, it is used to show the time course of the blockaveraged OD changes.


In [ ]:
from cedalion.plots import scalp_plot_gif

# configure the plot
data_ts = blockaverage.sel(wavelength=850, trial_type="FTapping/Right")
data_ts = data_ts.rename({"reltime": "time"})
geo3d = rec.geo3d
filename_scalp = "scalp_plot_ts"

# call plot function
scalp_plot_gif(
    data_ts,
    geo3d,
    filename=filename_scalp,
    time_range=(-5, 30, 0.5) * units.s,
    scl=(-0.01, 0.01),
    fps=6,
    optode_size=6,
    optode_labels=True,
    str_title="OD 850 nm",
)

In [ ]:
display_image(f"{filename_scalp}.gif")

### Image Space

#### Single-View Animations of Activitations on the Brain

The function `cedalion.plots.image_recon_view` allows to create an animated gif of image-space concentration changes projected on the brain.

In [ ]:
from cedalion.plots import image_recon_view

filename_view = 'image_recon_view'

X_ts = xr.concat([dC_brain.sel(trial_type="FTapping/Right"), dC_scalp.sel(trial_type="FTapping/Right")], dim="vertex")
X_ts = X_ts.rename({"reltime": "time"})
X_ts = X_ts.transpose("vertex", "chromo", "time")
X_ts = X_ts.assign_coords(is_brain=('vertex', Adot.is_brain.values))

scl = np.percentile(np.abs(X_ts.sel(chromo='HbO').values.reshape(-1)),99)
clim = (-scl,scl)

image_recon_view(
    X_ts,  # time series data; can be 2D (static) or 3D (dynamic)
    head,
    cmap='seismic',
    clim=clim,
    view_type='hbo_brain',
    view_position='left',
    title_str='HbO / uM',
    filename=filename_view,
    SAVE=True,
    time_range=(-5,30,0.5)*units.s,
    fps=6,
    geo3d_plot = geo3d_plot,
    wdw_size = (1024, 768)
)

In [ ]:
display_image(f"{filename_view}.gif")

Alternatively, we can just select a single time point and plot activity as a still image at that time. Note the different file suffix (.png).

In [ ]:
# selects the nearest time sample at t=4s in X_ts
X_ts_plot = X_ts.sel(time=4, method="nearest") # note: sel does not accept quantified units

filename_view = 'image_recon_view_still'

image_recon_view(
    X_ts_plot,  # time series data; can be 2D (static) or 3D (dynamic)
    head,
    cmap='seismic',
    clim=clim,
    view_type='hbo_brain',
    view_position='left',
    title_str='HbO / uM',
    filename=filename_view,
    SAVE=True,
    time_range=(-5,30,0.5)*units.s,
    fps=6,
    geo3d_plot = geo3d_plot,
    wdw_size = (1024, 768)
)

In [ ]:
display_image(f"{filename_view}.png")

#### Multi-View Animations of Activitations on the Brain

The function `cedalion.plots.image_recon_multi_view` shows the activity on the brain from all angles as still image or animated across time:

In [ ]:
from cedalion.plots import image_recon_multi_view

filename_multiview = 'image_recon_multiview'

# prepare data
X_ts = xr.concat([dC_brain.sel(trial_type="FTapping/Right"), dC_scalp.sel(trial_type="FTapping/Right")], dim="vertex")
X_ts = X_ts.rename({"reltime": "time"})
X_ts = X_ts.transpose("vertex", "chromo", "time")
X_ts = X_ts.assign_coords(is_brain=('vertex', Adot.is_brain.values))

scl = np.percentile(np.abs(X_ts.sel(chromo='HbO').values.reshape(-1)),99)
clim = (-scl,scl)


image_recon_multi_view(
    X_ts,  # time series data; can be 2D (static) or 3D (dynamic)
    head,
    cmap='seismic',
    clim=clim,
    view_type='hbo_brain',
    title_str='HbO / uM',
    filename=filename_multiview,
    SAVE=True,
    time_range=(-5,30,0.5)*units.s,
    fps=6,
    geo3d_plot = None, #  geo3d_plot
    wdw_size = (1024, 768)
)

In [ ]:
display_image(f"{filename_multiview}.gif")

#### Multi-View Animations of Activitations on the Scalp
This gives us activity on the scalp after recon from all angles as still image or across time

In [ ]:
from cedalion.plots import image_recon_multi_view

filename_multiview_scalp = 'image_recon_multiview_scalp'

# prepare data
X_ts = xr.concat([dC_brain.sel(trial_type="FTapping/Right"), dC_scalp.sel(trial_type="FTapping/Right")], dim="vertex")
X_ts = X_ts.rename({"reltime": "time"})
X_ts = X_ts.transpose("vertex", "chromo", "time")
X_ts = X_ts.assign_coords(is_brain=('vertex', Adot.is_brain.values))

scl = np.percentile(np.abs(X_ts.sel(chromo='HbO').values.reshape(-1)),99)
clim = (-scl,scl)


image_recon_multi_view(
    X_ts,  # time series data; can be 2D (static) or 3D (dynamic)
    head,
    cmap='seismic',
    clim=clim,
    view_type='hbo_scalp',
    title_str='HbO / uM',
    filename=filename_multiview_scalp,
    SAVE=True,
    time_range=(-5,30,0.5)*units.s,
    fps=6,
    geo3d_plot = geo3d_plot,
    wdw_size = (1024, 768)
)

In [ ]:
display_image(f"{filename_multiview_scalp}.gif")

### Parcel Space

The Schaefer Atlas <cite data-cite="Schaefer2018">(Schaefer, 2018)</cite> as implemented in Cedalion provides nearly 600 labels for different regions of the brain. Each vertex of the brain surface has its correspondng parcel label assigned as a coordinate.


In [ ]:
head.brain.vertices

Using parcel labels, vertices belonging to the same brain region can be easily grouped together with the `DataArray.groupby` function.

To obtain the average hemodynamic response in a parcel, the baseline-subtraced concentration changes of all vertices in a parcel are averaged. As baseline the first sample is used.

In [ ]:
# subtract baseline
baseline =  dC_brain.isel(reltime=0) # first sample along reltime dimension
dC_brain_blsubtracted = dC_brain - baseline

# average over parcels
avg_HbO = dC_brain_blsubtracted.sel(chromo="HbO").groupby('parcel').mean()

avg_HbR = dC_brain_blsubtracted.sel(chromo="HbR").groupby('parcel').mean()

display(dC_brain_blsubtracted.rename("dC_brain_blsubtracted"))
display(avg_HbO.rename("avg_HbO"))

The montage in this dataset covers only parts of the head. Consequently, many brain regions lack significant signal coverage due to the absence of optodes. 

To focus on relevant regions, a subset of parcels from the somatosensory and motor regions in both hemispheres is selected.

In [ ]:
selected_parcels = [
    "SomMotA_1_LH", "SomMotA_3_LH", "SomMotA_4_LH", "SomMotA_5_LH", "SomMotA_9_LH", "SomMotA_10_LH",
    "SomMotA_1_RH", "SomMotA_2_RH", "SomMotA_3_RH", "SomMotA_4_RH", "SomMotA_6_RH", "SomMotA_7_RH"
]

The following plot visualizes the montage and the selected parcels:

In [ ]:
# map parcel labels to colors
parcel_colors = {
    parcel: p.cm.jet(i / (len(selected_parcels) - 1))
    for i, parcel in enumerate(selected_parcels)
}

# assign colors to vertices
b = cdc.VTKSurface.from_trimeshsurface(head.brain)
b = pv.wrap(b.mesh)
b["parcels"] = np.asarray([
    parcel_colors.get(parcel, (0.8, 0.8, 0.8, .3))
    for parcel in head.brain.vertices.parcel.values
])

plt = pv.Plotter()

plt.add_mesh(
    b,
    scalars="parcels",
    rgb=True,
    smooth_shading=False
)

cedalion.plots.plot_labeled_points(plt, geo3d_plot)

legends = [a for a in parcel_colors.items()]
plt.add_legend(labels= legends, face='o', size=(0.3,0.3))

cog = head.brain.vertices.mean("label")
cog = cog.pint.dequantify().values

plt.camera.position = cog + [0,0,400]
plt.camera.focal_point = cog
plt.camera.up = [0,1,0]
plt.reset_camera()

plt.show()

Plot averaged time traces in each parcel for the 'FTapping/Right' and 'FTapping/Left' conditions:

In [ ]:
f,ax = p.subplots(2,6, figsize=(20,5))
ax = ax.flatten()
for i_par, par in enumerate(selected_parcels):
    ax[i_par].plot(avg_HbO.sel(parcel = par, trial_type = "FTapping/Right").reltime, avg_HbO.sel(parcel = par, trial_type = "FTapping/Right").values, "r", lw=2, ls='-')
    ax[i_par].plot(avg_HbR.sel(parcel = par, trial_type = "FTapping/Right").reltime, avg_HbR.sel(parcel = par, trial_type = "FTapping/Right").values, "b", lw=2, ls='-')

    ax[i_par].grid(1)
    ax[i_par].set_title(par, color=parcel_colors[par])
    ax[i_par].set_ylim(-.05, .2)

p.suptitle("Parcellations: HbO: r | HbR: b | FTapping/Right", y=1)
p.tight_layout()

f,ax = p.subplots(2,6, figsize=(20,5))
ax = ax.flatten()
for i_par, par in enumerate(selected_parcels):
    ax[i_par].plot(avg_HbO.sel(parcel = par, trial_type = "FTapping/Left").reltime, avg_HbO.sel(parcel = par, trial_type = "FTapping/Left").values, "r", lw=2, ls='-')
    ax[i_par].plot(avg_HbR.sel(parcel = par, trial_type = "FTapping/Left").reltime, avg_HbR.sel(parcel = par, trial_type = "FTapping/Left").values, "b", lw=2, ls='-')

    ax[i_par].grid(1)
    ax[i_par].set_title(par, color=parcel_colors[par])
    ax[i_par].set_ylim(-.05, .2)

p.suptitle("Parcellations: HbO: r | HbR: b | FTapping/Left", y=1)
p.tight_layout()